## Ampl imlementation on operations-rooom data

*Find the tutorial on the following page:*<br>
https://ampl.com/api/extra/python_quickstart.html

In [56]:
from pathlib import Path

import os
directory=os.getcwd()
print(directory)



/Users/kultiginbozdemir/Documents/GitHub/or_club/Google_ortools


In [57]:
from __future__ import print_function
import pandas as pd  
from bokeh.layouts import row
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [58]:
import amplpy
from amplpy import AMPL, Environment, DataFrame

In [59]:
AMPL(Environment('/Applications/ampl'))

In [60]:
import os
os.environ['PATH'] += os.pathsep + '/Applications/ampl'
ampl = AMPL()


In [61]:
ampl.setOption('solver', 'gurobi')

In [62]:
ampl.eval('''
set FOOD;
set NUTR;
# Parameters
param cost { FOOD } > 0;
param f_min { FOOD } >= 0;
param f_max {j in FOOD } >= f_min [j];
param n_min { NUTR } >= 0;
param n_max {i in NUTR } >= n_min [i];
param amt {NUTR , FOOD } >= 0;
# Variables
var Buy {j in FOOD } >= f_min [j], <= f_max [j];
# Objective
minimize total_cost : sum {j in FOOD } cost [j] * Buy[j];
# Contraints
subject to diet {i in NUTR }:
    n_min [i] <= sum {j in FOOD } amt[i,j] * Buy[j] <= n_max [i];
''')

In [63]:
foods = ['BEEF', 'CHK', 'FISH', 'HAM', 'MCH', 'MTL', 'SPG', 'TUR']
nutrients = ['A', 'C', 'B1', 'B2', 'NA', 'CAL']

In [64]:
ampl.getSet('FOOD').setValues(foods)
ampl.getSet('NUTR').setValues(nutrients)

In [65]:
ampl.setData(DataFrame(
    index=[('FOOD', foods)], 
    columns=[
        ('cost', [3.59, 2.59, 2.29, 2.89, 1.89, 1.99, 1.99, 2.49]), 
        ('f_min', [2, 2, 2, 2, 2, 2, 2, 2]), 
        ('f_max', [10, 10, 10, 10, 10, 10, 10, 10])
    ]
))


In [66]:
df = pd.DataFrame({
    'n_min': [700, 700, 700, 700, 0, 16000], 
    'n_max': [20000, 20000, 20000, 20000, 50000, 24000]
}, 
    index=nutrients
)
ampl.setData(DataFrame.fromPandas(df))

In [73]:
amounts = [
    [ 60,    8,   8,  40,   15,  70,   25,   60],
    [ 20,    0,  10,  40,   35,  30,   50,   20],
    [ 10,   20,  15,  35,   15,  15,   25,   15],
    [ 15,   20,  10,  10,   15,  15,   15,   10],
    [928, 2180, 945, 278, 1182, 896, 1329, 1397],
    [295,  770, 440, 430,  315, 400,  379,  450]
]
df = DataFrame(('NUTR', 'FOOD'), 'amt')
df.setValues({
    (nutrient, food): amounts[i][j]
    for i, nutrient in enumerate(nutrients)
    for j, food in enumerate(foods)
})
ampl.setData(df)
print(df)

    NUTR         FOOD     |     amt     
    'A'           'BEEF'    |      60     
    'A'           'CHK'     |      8      
    'A'           'FISH'    |      8      
    'A'           'HAM'     |      40     
    'A'           'MCH'     |      15     
    'A'           'MTL'     |      70     
    'A'           'SPG'     |      25     
    'A'           'TUR'     |      60     
    'C'           'BEEF'    |      20     
    'C'           'CHK'     |      0      
    'C'           'FISH'    |      10     
    'C'           'HAM'     |      40     
    'C'           'MCH'     |      35     
    'C'           'MTL'     |      30     
    'C'           'SPG'     |      50     
    'C'           'TUR'     |      20     
    'B1'          'BEEF'    |      10     
    'B1'          'CHK'     |      20     
    'B1'          'FISH'    |      15     
    'B1'          'HAM'     |      35     
    'B1'          'MCH'     |      15     
    'B1'          'MTL'     |      15     
    'B1'     

In [69]:
ampl.solve()

Gurobi 9.1.1: optimal solution; objective 119.9897589
5 simplex iterations


In [70]:
ampl.getVariable('Buy').getValues().toPandas()

,Buy.val
BEEF,5.226933
CHK,2.000000
FISH,2.000000
HAM,10.000000
MCH,10.000000
MTL,10.000000
SPG,9.439734
TUR,2.000000


In [45]:
totalcost = ampl.getObjective('total_cost')
print("Objective is:", totalcost.value())

Objective is: 119.98975893599335


In [46]:
cost = ampl.getParameter('cost')
cost.setValues({'BEEF': 5.01, 'HAM': 4.55})
print("Increased costs of beef and ham.")

Increased costs of beef and ham.


In [47]:
ampl.solve()

Gurobi 9.1.1: optimal solution; objective 144.0120033
5 simplex iterations


In [48]:
print("New objective value:", totalcost.value())

New objective value: 144.01200332502077


In [49]:
Buy = ampl.getVariable('Buy')
print("Buy['BEEF'].val = {}".format(Buy['BEEF'].value()))

Buy['BEEF'].val = 5.226932668329175


In [50]:
diet = ampl.getConstraint('diet')
for nutr in nutrients:
    print("diet['{}'].dual = {}".format(nutr, diet[nutr].dual()))


diet['A'].dual = 0.0
diet['C'].dual = 0.0
diet['B1'].dual = 0.0
diet['B2'].dual = 0.7999285120532003
diet['NA'].dual = -0.007531172069825435
diet['CAL'].dual = 0.0


In [51]:
rows = [tuple(row) for row in Buy.getValues()]
factors = [index for index, value in rows]
x = [value for index, value in rows]

dot = figure(
    title="Categorical Dot Plot", tools='', toolbar_location=None,
    y_range=factors, x_range=[0,12]
)

dot.segment(0, factors, x, factors, line_width=2, line_color='green')
dot.circle(x, factors, size=15, fill_color='orange', line_color='green', line_width=3)
show(dot)